# Universidad del Desarrollo
## Magister Data Science - Macroeconomía
### Grupo N°2: (Región de los Lagos)
* Jose Pedro
* Cristian
* Paul
* Max
* Hernan

## Calculo IPC

**Objetivo**

El objetivo de este notebook es el calculo de IPC según el siguiente algortimo.



$ 
\begin{equation*}
IPC_t   = |\sum_{k=1}^N \frac{p_i^t}{p_i^0} w_i| * 100
\end{equation*} 
$

$ 
\begin{equation*} 
p_i^t = \prod_{j=1}^n (\frac{v_j^t}{v_j^{t-1}})^{1/n} 
\end{equation*} 
$

Donde:
* $ p_i^t $ = Precio del producto i en el momento t
* $ p_i^0 $ = Precio del producto 0 en el mes base **(2014/08=100)**
* $ w_i   $ = Ponderación del producto i en la canasta
* $ v_j^t $ = Precio de la variaedad j en el momento t
* N = Numero de productos
* n = Numero de variedades

Dados los siguientes supuestos:
* n = 2
* t = 2014/09

Entonces:

P = ([ p_var1(2014-09) / p_var1(2014-08) ]$**$(1/2)) $*$ ([ p_var2(2014-09) / p_var2(2014-08) ]$**$(1/2))

In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime

from dateutil.relativedelta import relativedelta

%matplotlib inline

### 1. Se carga RUT con los productos

In [2]:
df = pd.read_csv('output/canasta_ipc.csv')

In [3]:
df.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08,Osorno,Aceite,Maravilla,1558.2966
1,2014-08,Osorno,Aceite,Vegetal,1410.9042
2,2014-08,Puerto Montt,Aceite,Maravilla,1516.6203
3,2014-08,Puerto Montt,Aceite,Vegetal,1357.3113
4,2014-09,Frutillar,Aceite,Maravilla,1680.2222


In [4]:
df.Producto.value_counts()

Carnes       10631
Hortaliza     8078
Frutas        5867
Lacteos       2239
Pan           1868
Legumbre      1205
Cereales       900
Harina         450
Aceite         450
Huevos         449
Azucar         225
Name: Producto, dtype: int64

### 2.- Transformamos Monento a un tipo de dato fecha para iterar sobre el periodo

In [5]:
df["Momento"] = pd.to_datetime(df["Momento"])

In [6]:
df.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08-01,Osorno,Aceite,Maravilla,1558.2966
1,2014-08-01,Osorno,Aceite,Vegetal,1410.9042
2,2014-08-01,Puerto Montt,Aceite,Maravilla,1516.6203
3,2014-08-01,Puerto Montt,Aceite,Vegetal,1357.3113
4,2014-09-01,Frutillar,Aceite,Maravilla,1680.2222


* Años base:

In [7]:
base = datetime.date(2015, 4, 1)

### 3. Creamos una funcion para restar un mes al periodo de estudio

In [8]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime.date(year, month, day)

### 4. Creamos función para calculo de la serie factores de producto

In [9]:
def GetSerieProducto(df_input, group):
    v = []
    for name, g in group: 
        
        #print(name)
        factor = 1
        n = g.Promedio.count()

        if g.Momento.iloc[0].date() > base:
            for row_num, row in g.iterrows(): 

                Pro1 = row["Promedio"]
                Mom2 = add_months(row["Momento"], -1)

                Pro2 = df_input[(df_input.Sector==row["Sector"]) & (df_input.Momento == Mom2) & (df_input.Variedad == row["Variedad"])]["Promedio"].iloc[0]

                factor *= (Pro1/Pro2)**(1/n)

        name = name + (factor,)  
        v.append(name)
    
    return pd.DataFrame(v, columns=['Momento', 'Sector', 'Producto', 'Factor'])

### 5. Aceite

In [10]:
df_aceite = df[df.Producto=="Aceite"]
group = df_aceite[df_aceite.Momento > "2015-03"].groupby(['Momento', 'Sector', 'Producto'])

In [11]:
dfs_aceite = GetSerieProducto(df_aceite, group)

In [12]:
dfs_aceite.sample(3)

,Momento,Sector,Producto,Factor
89,2016-09-01,Puerto Varas,Aceite,0.993567
68,2016-05-01,Puerto Montt,Aceite,0.958993
131,2017-06-01,Llanquihue,Aceite,1.007693


### 6. Harina

In [13]:
df_harina = df[df.Producto=="Harina"]
group = df_harina[df_harina.Momento > "2015-03"].groupby(['Momento', 'Sector', 'Producto'])

In [14]:
dfs_harina = GetSerieProducto(df_harina, group)

In [15]:
dfs_harina.sample(3)

,Momento,Sector,Producto,Factor
120,2017-04-01,Frutillar,Harina,1.091499
22,2015-08-01,Osorno,Harina,0.991026
48,2016-01-01,Puerto Montt,Harina,0.998421


### 7. Cereales

In [20]:
df_cereales = df[df.Producto=="Cereales"]
group = df_cereales[df_cereales.Momento > "2015-03"].groupby(['Momento', 'Sector', 'Producto'])

In [21]:
dfs_cereales = GetSerieProducto(df_cereales, group)

In [22]:
dfs_cereales.sample(3)

,Momento,Sector,Producto,Factor
176,2018-03-01,Llanquihue,Cereales,1.007128
43,2015-12-01,Puerto Montt,Cereales,0.995591
162,2017-12-01,Osorno,Cereales,0.990524


### 8. Azucar

In [39]:
df_azucar = df[df.Producto=="Azucar"]

In [40]:
df_azucar.isnull().sum()

Momento       0
Sector        0
Producto      0
Variedad    225
Promedio      0
dtype: int64

In [41]:
df_azucar["Variedad"].fillna("Azucar normal", inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [42]:
group = df_azucar[df_azucar.Momento > "2015-03"].groupby(['Momento', 'Sector', 'Producto'])

In [43]:
dfs_azucar = GetSerieProducto(df_azucar, group)

In [44]:
dfs_azucar.sample(3)

,Momento,Sector,Producto,Factor
185,2018-05-01,Frutillar,Azucar,0.985916
31,2015-10-01,Llanquihue,Azucar,1.000000
62,2016-04-01,Osorno,Azucar,0.996295


### Creamos funcion para calculo del IPC